# Amazon Bedrock Model Catalog Demo - CRIS Data Access

This notebook demonstrates how to use the **BedrockModelCatalog** to access Amazon Bedrock Cross-Region Inference (CRIS) model information.

## Overview

The BedrockModelCatalog provides:
- Unified API-based access to model and CRIS data
- Automatic data fetching from AWS Bedrock APIs
- Configurable caching strategies (FILE, MEMORY, NONE)
- Bundled fallback data for offline scenarios
- Comprehensive model access information including inference profiles
- Flexible model name resolution (supports aliases and variations)

## What's New?

This notebook has been updated to use the new **BedrockModelCatalog** which replaces the deprecated `CRISManager`. Key improvements:
- ✅ API-only data retrieval (no HTML parsing)
- ✅ Automatic initialization (no manual refresh calls)
- ✅ Unified model and CRIS data access
- ✅ Better error handling and fallback mechanisms
- ✅ Lambda-friendly design with configurable caching

## Setup and Imports

First, let's import the required modules and configure logging.

In [ ]:
# Import required modules
import sys
import logging
from pathlib import Path
from datetime import datetime
import json

# Add the src directory to the Python path
sys.path.append('../src')

# Import the new BedrockModelCatalog
from bestehorn_llmmanager.bedrock.catalog import BedrockModelCatalog, CacheMode
from bestehorn_llmmanager.bedrock.exceptions.llm_manager_exceptions import CatalogUnavailableError

# Configure logging to see what's happening
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ BedrockModelCatalog imported successfully!")

## 1. Basic Usage - Initialize the Catalog

Let's initialize the BedrockModelCatalog with `force_refresh=True` to fetch fresh data from AWS APIs.

**Note**: The catalog automatically handles:
- Fetching data from AWS Bedrock APIs
- Caching for performance
- Fallback to bundled data if APIs are unavailable

In [ ]:
# Initialize the catalog with force refresh for demonstration
# This ensures we get fresh data from AWS APIs
print("🔄 Initializing BedrockModelCatalog with fresh data...")

try:
    catalog = BedrockModelCatalog(
        force_refresh=True,  # Always fetch fresh data for demo
        timeout=60,          # Longer timeout for reliability
        fallback_to_bundled=True  # Fallback if API fails
    )
    
    print("✅ BedrockModelCatalog initialized successfully!")
    print(f"   Cache mode: {catalog.cache_mode.value}")
    print(f"   Catalog loaded: {catalog.is_catalog_loaded}")
    
except CatalogUnavailableError as e:
    print(f"❌ Failed to initialize catalog: {e}")
    print("💡 Check your AWS credentials and network connectivity")
    raise

## 2. Display Catalog Metadata

Let's examine the metadata to understand where the data came from and when it was retrieved.

In [ ]:
# Get catalog metadata
metadata = catalog.get_catalog_metadata()

print("📊 Catalog Metadata:")
print("=" * 60)
print(f"   Source: {metadata.source.value}")
print(f"   Retrieved: {metadata.retrieval_timestamp}")
print(f"   Regions Queried: {len(metadata.api_regions_queried)}")
print(f"   Region List: {', '.join(metadata.api_regions_queried)}")

if metadata.cache_file_path:
    print(f"   Cache File: {metadata.cache_file_path}")
    
if metadata.bundled_data_version:
    print(f"   Bundled Version: {metadata.bundled_data_version}")

## 3. List All Available Models

Let's see what models are available in the catalog.

In [ ]:
# List all models
all_models = catalog.list_models()

print(f"📋 Total Models Available: {len(all_models)}")
print("\nModel Names:")
print("=" * 60)

for i, model in enumerate(all_models[:10], 1):  # Show first 10
    print(f"   {i}. {model.model_name}")
    
if len(all_models) > 10:
    print(f"   ... and {len(all_models) - 10} more models")

## 4. Get Model Information with CRIS Data

Let's retrieve detailed information for specific models, including their inference profile data.

In [ ]:
# Example models to query
example_models = [
    ("Claude 3 Haiku", "us-east-1"),
    ("Claude 3.5 Sonnet", "us-west-2"),
    ("Nova Lite", "us-east-1")
]

print("🔍 Model Information with CRIS Data:")
print("=" * 60)

for model_name, region in example_models:
    print(f"\n📋 Model: {model_name} in {region}")
    print("-" * 40)
    
    # Get model info using the new API
    model_info = catalog.get_model_info(model_name=model_name, region=region)
    
    if model_info:
        print(f"   ✅ Model Available")
        print(f"   Model ID: {model_info.model_id}")
        print(f"   Inference Profile: {model_info.inference_profile_id or 'N/A'}")
        print(f"   Access Method: {model_info.access_method.value}")
        print(f"   Streaming: {'✅' if model_info.supports_streaming else '❌'}")
        print(f"   Input Modalities: {', '.join(model_info.input_modalities)}")
        print(f"   Output Modalities: {', '.join(model_info.output_modalities)}")
    else:
        print(f"   ❌ Model not found or not available in {region}")

## 5. Filter Models by Provider

Let's filter models by provider to see what's available from specific vendors.

In [ ]:
# Filter by provider
providers_to_check = ["Anthropic", "Amazon", "Meta"]

print("🏢 Models by Provider:")
print("=" * 60)

for provider in providers_to_check:
    provider_models = catalog.list_models(provider=provider)
    print(f"\n{provider}: {len(provider_models)} models")
    
    # Show first 5 models from this provider
    for model in provider_models[:5]:
        print(f"   • {model.model_name}")
        
    if len(provider_models) > 5:
        print(f"   ... and {len(provider_models) - 5} more")

## 6. Check Model Availability by Region

Let's check which models are available in specific regions.

In [ ]:
# Check availability in different regions
regions_to_check = ["us-east-1", "us-west-2", "eu-west-1"]
models_to_check = ["Claude 3 Haiku", "Claude 3.5 Sonnet", "Nova Lite"]

print("🌍 Model Availability by Region:")
print("=" * 60)

for model_name in models_to_check:
    print(f"\n📋 {model_name}:")
    
    for region in regions_to_check:
        is_available = catalog.is_model_available(model_name=model_name, region=region)
        status = "✅" if is_available else "❌"
        print(f"   {status} {region}")

## 7. Filter Models by Region

Let's see all models available in a specific region.

In [ ]:
# Filter by region
target_region = "us-east-1"

print(f"🌍 Models Available in {target_region}:")
print("=" * 60)

region_models = catalog.list_models(region=target_region)
print(f"\nTotal: {len(region_models)} models")

# Group by provider
by_provider = {}
for model in region_models:
    provider = model.provider
    if provider not in by_provider:
        by_provider[provider] = []
    by_provider[provider].append(model.model_name)

print("\nBreakdown by Provider:")
for provider, models in sorted(by_provider.items()):
    print(f"   {provider}: {len(models)} models")

## 8. Filter Streaming-Capable Models

Let's find all models that support streaming responses.

In [ ]:
# Filter streaming-capable models
print("🌊 Streaming-Capable Models:")
print("=" * 60)

streaming_models = catalog.list_models(streaming_only=True)
print(f"\nTotal: {len(streaming_models)} streaming-capable models")

# Show first 10
print("\nExamples:")
for i, model in enumerate(streaming_models[:10], 1):
    print(f"   {i}. {model.model_name} ({model.provider})")
    
if len(streaming_models) > 10:
    print(f"   ... and {len(streaming_models) - 10} more")

## 9. Combine Multiple Filters

Let's combine multiple filters to find specific models.

In [ ]:
# Combine filters: Anthropic models in us-west-2 that support streaming
print("🔍 Combined Filter Example:")
print("   Provider: Anthropic")
print("   Region: us-west-2")
print("   Streaming: Required")
print("=" * 60)

filtered_models = catalog.list_models(
    provider="Anthropic",
    region="us-west-2",
    streaming_only=True
)

print(f"\nFound: {len(filtered_models)} models")

for model in filtered_models:
    print(f"   • {model.model_name}")

## 10. Understanding Access Methods

Models can be accessed through different methods:
- **direct**: Direct model ID access in the region
- **regional_cris**: Regional inference profile (e.g., us.model-id)
- **global_cris**: Global inference profile (e.g., model-id without region prefix)

Let's examine the access methods for different models.

In [ ]:
# Examine access methods
print("🔑 Model Access Methods:")
print("=" * 60)

example_models = ["Claude 3 Haiku", "Claude 3.5 Sonnet", "Nova Lite"]
example_region = "us-east-1"

for model_name in example_models:
    model_info = catalog.get_model_info(model_name=model_name, region=example_region)
    
    if model_info:
        print(f"\n📋 {model_name}:")
        print(f"   Model ID: {model_info.model_id}")
        print(f"   Access Method: {model_info.access_method.value}")
        
        if model_info.inference_profile_id:
            print(f"   Inference Profile: {model_info.inference_profile_id}")
            print(f"   💡 Use inference profile for cross-region routing")
        else:
            print(f"   💡 Use model ID for direct access")

## 11. Troubleshooting

Common issues and solutions when working with BedrockModelCatalog.

### Import Errors

If you encounter import errors:

```python
# ❌ ImportError: No module named 'bestehorn_llmmanager'
```

**Solutions**:
1. Ensure you're running from the `notebooks/` directory
2. Check that `sys.path.append('../src')` is executed
3. Verify the package is installed: `pip install -e .` from project root

### API Timeout Errors

If API fetching times out:

```python
# Increase timeout and enable fallback
catalog = BedrockModelCatalog(
    force_refresh=True,
    timeout=120,  # Longer timeout
    fallback_to_bundled=True  # Use bundled data if API fails
)
```

### Cache Permission Errors

If you can't write to the cache directory:

```python
# Use memory-only caching
catalog = BedrockModelCatalog(
    cache_mode=CacheMode.MEMORY,
    force_refresh=True
)
```

### Model Not Found

If a model isn't found:

```python
# Check available models
all_models = catalog.list_models()
model_names = [m.model_name for m in all_models]
print("Available models:", model_names)

# Try different name variations
# The catalog supports flexible name matching
variations = [
    "Claude 3 Haiku",
    "claude-3-haiku",
    "anthropic.claude-3-haiku-20240307-v1:0"
]
```

## 12. Summary and Best Practices

Summary of BedrockModelCatalog capabilities and usage recommendations.

In [ ]:
print("📋 BedrockModelCatalog Summary:")
print("=" * 60)

print("\n✅ Key Features Demonstrated:")
features = [
    "Unified API-based model and CRIS data access",
    "Automatic initialization with force_refresh option",
    "Comprehensive model information including inference profiles",
    "Flexible filtering by provider, region, and streaming support",
    "Model availability checking across regions",
    "Access method detection (direct, regional_cris, global_cris)",
    "Catalog metadata for source and freshness tracking",
    "Configurable caching strategies (FILE, MEMORY, NONE)",
    "Bundled fallback data for offline scenarios",
    "Flexible model name resolution (aliases and variations)"
]

for i, feature in enumerate(features, 1):
    print(f"   {i}. {feature}")

print("\n💡 Best Practices:")
practices = [
    "Use force_refresh=True for demos to ensure fresh data",
    "Enable fallback_to_bundled=True for reliability",
    "Handle CatalogUnavailableError exceptions properly",
    "Use appropriate cache modes for your use case",
    "Check catalog metadata to understand data source",
    "Use list_models() with filters for efficient queries",
    "Leverage flexible name matching for user-friendly queries",
    "Monitor access methods for optimal API usage"
]

for i, practice in enumerate(practices, 1):
    print(f"   {i}. {practice}")

print("\n🎯 Use Cases:")
use_cases = [
    "Model availability checking across regions",
    "Cross-region inference setup with CRIS profiles",
    "Provider-specific model discovery",
    "Streaming capability verification",
    "Multi-region deployment planning",
    "Cost optimization through region selection",
    "Disaster recovery and failover configuration"
]

for i, use_case in enumerate(use_cases, 1):
    print(f"   {i}. {use_case}")

print("\n📚 Additional Resources:")
resources = [
    "Migration Guide: docs/MIGRATION_GUIDE.md",
    "API Reference: docs/forLLMConsumption.md",
    "Code Examples: examples/catalog_*.py",
    "README: README.md (BedrockModelCatalog section)"
]

for resource in resources:
    print(f"   • {resource}")

print("\n🎉 BedrockModelCatalog Demo Complete!")
print("   Ready for production use with comprehensive model catalog management.")